# 初めに
https://www.youtube.com/watch?v=KldiyIiY1ao
こちらの動画よりコードをフォークしたものです。

# ライブラリのインポート

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# データの確認

In [3]:
data_root = './data'

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
    transforms.Lambda(lambda x : x.view(-1))
])

train_set = datasets.MNIST(
    root = data_root,
    train = True,
    download = True,
    transform = transform
)

test_set = datasets.MNIST(
    root = data_root,
    train = False,
    download = True,
    transform = transform
)

In [4]:
len(train_set)

60000

In [5]:
image, label = train_set[0]
print('Image type:', type(image))
print('Image shape:', image.shape)

Image type: <class 'torch.Tensor'>
Image shape: torch.Size([784])


In [6]:
image, label = test_set[0]
print(len(test_set))
print('Image type:', type(image))
print('Image shape:', image.shape)

10000
Image type: <class 'torch.Tensor'>
Image shape: torch.Size([784])


In [7]:
print('min: ', image.data.min())
print('max: ', image.data.max())

min:  tensor(-1.)
max:  tensor(1.)


In [8]:
"""
image, label = train_set[0]
plt.figure(figsize=(2, 2))
plt.title(f'{label}')
plt.imshow(image, cmap = 'gray_r')
plt.show()
"""

"\nimage, label = train_set[0]\nplt.figure(figsize=(2, 2))\nplt.title(f'{label}')\nplt.imshow(image, cmap = 'gray_r')\nplt.show()\n"

In [9]:
batch_size = 500

train_loader = DataLoader(
    train_set,
    batch_size = batch_size,
    shuffle = True
)

test_loader = DataLoader(
    test_set,
    batch_size = batch_size,
    shuffle = True
)

In [10]:
for image, label in train_loader:
  break

image.shape

torch.Size([500, 784])

In [11]:
n_input = 784
n_output = 10
n_hidden = 128

# モデルのクラスの定義

In [22]:
class Net(nn.Module):
  def __init__(self, n_input, n_output, n_hiddern):
    super().__init__()
    self.l1 = nn.Linear(n_input, n_hidden)
    self.l2 = nn.Linear(n_hidden, n_output)
    self.relu = nn.ReLU(inplace=True)

  def forward(self, x):
    x1 = self.l1(x)
    x2 = self.relu(x1)
    x3 = self.l2(x2)

    return x3

In [23]:
net = Net(n_input, n_output, n_hidden).to(device)

In [24]:
criterion = nn.CrossEntropyLoss()

In [25]:
lr = 0.01
optimizer = optim.SGD(net.parameters(), lr = lr)

In [26]:
history = np.zeros((0, 5))
history

array([], shape=(0, 5), dtype=float64)

# 学習

In [27]:
from tqdm.notebook import tqdm

In [33]:
num_epoch = 10

for epoch in range(num_epoch):
  train_acc, train_loss = 0, 0
  val_acc, val_loss = 0, 0
  n_train, n_test = 0, 0

  for inputs, labels in tqdm(train_loader):
    n_train += len(labels)

    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()

    optimizer.step()
    predicted = torch.max(outputs, 1)[1]
    
    train_loss += loss.item()
    train_acc += (predicted == labels).sum().item()
  
  for inputs_test, labels_test in test_loader:
    n_test += len(labels_test)

    inputs_test = inputs_test.to(device)
    labels_test = labels_test.to(device)

    output_test = net(inputs_test)
    loss_test = net(inputs_test)

    predicted_test = torch.max(output_test, 1)[1]
    
    val_loss += loss.item()
    val_acc += (predicted_test == labels_test).sum().item()

  train_acc = train_acc / n_train
  val_acc = val_acc / n_test
  train_loss = train_loss * batch_size / n_train
  val_loss = val_loss * batch_size / n_test
  print(f'Epoch [{epoch+1}/{num_epoch}], loss: {train_loss:.5f}, acc: {train_acc:.5f} val_loss: {val_loss:.5f}, val_acc:{val_acc:.5f}')
  items = np.array([epoch+1, train_loss, train_acc, val_loss, val_acc])
  history = np.vstack((history, items))

  0%|          | 0/120 [00:00<?, ?it/s]

Epoch [1/10], loss: 0.35758, acc: 0.89855 val_loss: 0.36176, val_acc:0.90400


  0%|          | 0/120 [00:00<?, ?it/s]

Epoch [2/10], loss: 0.34993, acc: 0.90035 val_loss: 0.39612, val_acc:0.90350


  0%|          | 0/120 [00:00<?, ?it/s]

...

Epoch [8/10], loss: 0.31849, acc: 0.90832 val_loss: 0.32744, val_acc:0.91300


  0%|          | 0/120 [00:00<?, ?it/s]

Epoch [9/10], loss: 0.31435, acc: 0.90930 val_loss: 0.28139, val_acc:0.91300


  0%|          | 0/120 [00:00<?, ?it/s]

Epoch [10/10], loss: 0.31076, acc: 0.91072 val_loss: 0.28892, val_acc:0.91500


# 学習曲線の可視化

In [34]:
#損失と精度の確認
print(f'初期状態: 損失: {history[0, 3]:.5f} 精度: {history[0, 4]:.5f}')
print(f'最終状態: 損失: {history[-1, 3]:.5f} 精度: {history[-1, 4]:.5f}')

初期状態: 損失: 0.85246 精度: 0.84490
最終状態: 損失: 0.28892 精度: 0.91500


In [37]:
history

array([[ 1.        ,  1.02105186,  0.80861667,  0.85246229,  0.8449    ],
       [ 2.        ,  0.69578486,  0.84698333,  0.60477811,  0.8629    ],
       [ 3.        ,  0.56388551,  0.8634    ,  0.4952997 ,  0.8744    ],
       [ 4.        ,  0.49588354,  0.87228333,  0.49344376,  0.8825    ],
       [ 5.        ,  0.45433102,  0.87961667,  0.47254622,  0.8889    ],
       [ 6.        ,  0.42613558,  0.88468333,  0.42679307,  0.8919    ],
       [ 7.        ,  0.4054591 ,  0.88855   ,  0.39810017,  0.8954    ],
       [ 8.        ,  0.38968236,  0.8916    ,  0.34256145,  0.8979    ],
       [ 9.        ,  0.37702711,  0.89438333,  0.36748394,  0.8987    ],
       [10.        ,  0.36656351,  0.89686667,  0.40835944,  0.9008    ],
       [ 1.        ,  0.35757895,  0.89855   ,  0.36176056,  0.904     ],
       [ 2.        ,  0.34992839,  0.90035   ,  0.39611709,  0.9035    ],
       [ 3.        ,  0.3432946 ,  0.90183333,  0.316737  ,  0.9064    ],
       [ 4.        ,  0.33738706,  0.9